In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
import ast

filename = 'movie_name.txt'
with open(filename, 'r', encoding='utf-8') as file:
    content = file.read()
movie_name = ast.literal_eval(content)

## Pull All Comments

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import requests

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = dev

youtube = build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

In [ ]:
def fetch_all_results(query, max_results_per_page=49):
    results = []
    next_page_token = None

    while True:
        # Fetch the search results
        response = youtube.search().list(
            q=query,
            part='snippet',
            order='relevance',
            maxResults=max_results_per_page,
            type='video',
            videoDuration='short',
            pageToken=next_page_token
        ).execute()

        # Add the current page's results
        results.extend(response.get('items', []))

    return results

def filter_by_description(videos, keyword):
    filtered_videos = [video for video in videos if keyword in video['snippet']['description']]
    return filtered_videos

# Fetch all results
def getVideoId(movie_name):
  all_videos={}
  for movieName in movie_name :
    query = f'{movieName} 예고편 -1차 -2차 -3차'
    result=fetch_all_results(query,1)
    all_videos[movieName]={'VideoId':result['Id']['videoId']}
    #all_videos.update({'VideoId':result['Id']['videoId'],'moiveName':movieName})

  return all_videos

def get_video_details(movie_name):
    # Make the API request
    all_videos=getVideoId(movie_name)

    movie_names = []
    video_ids = []

    for movie_name, details in all_videos.items():
      movie_names.append(movie_name)  # Extract movie name
      video_ids.append(details['VideoId'])

    response = youtube.videos().list(
        part='snippet,statistics',
        id=','.join(video_ids)
    ).execute()

    # Extract details
    video_details = []
    index=0
    for video in response.get('items', []):
        video_id = video['id']
        statistics = video['statistics']
        video_details.append({
            'moive_name': movie_names[index],
            'viewCount': statistics.get('viewCount', '0'),
            'likeCount': statistics.get('likeCount', '0'),
            'commentCount' : statistics.get('commentCount','0')
        })
        index++

    return video_details









In [ ]:
movie_details=get_video_details(movie_name)

In [ ]:
df = pd.DataFrame(movie_details, columns=['movie_name', 'view_count', 'like_count', 'comment_count'])


In [ ]:
import googleapiclient.discovery
import pandas as pd

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = dev

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part="snippet",
    videoId="-GJgqIJsTME",
    maxResults=100,
    publishedBefore="1970-01-01T00:00:00Z"
)

comments = []

# Execute the request.
response = request.execute()
response.keys()





dict_keys(['kind', 'etag', 'nextPageToken', 'pageInfo', 'items'])

In [ ]:
# Get the comments from the response.
for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    public = item['snippet']['isPublic']
    comments.append([
        comment['authorDisplayName'],
        comment['publishedAt'],
        comment['likeCount'],
        comment['textOriginal'],
        public
    ])

while (1 == 1):
  try:
   nextPageToken = response['nextPageToken']
  except KeyError:
   break
  nextPageToken = response['nextPageToken']
  # Create a new request object with the next page token.
  nextRequest = youtube.commentThreads().list(part="snippet", videoId="-GJgqIJsTME", maxResults=100, pageToken=nextPageToken)
  # Execute the next request.
  response = nextRequest.execute()
  # Get the comments from the next response.
  for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    public = item['snippet']['isPublic']
    comments.append([
        comment['authorDisplayName'],
        comment['publishedAt'],
        comment['likeCount'],
        comment['textOriginal'],
        public
    ])

df = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text','public'])
df.info()

In [ ]:
response['items'][0]

{'kind': 'youtube#commentThread',
 'etag': 'vmLOmjQOxNPBJcA6PKJnB9Yl3g0',
 'id': 'Ugz64fenKxBqRkeabUB4AaABAg',
 'snippet': {'channelId': 'UC2utQmYluWGlhV-W0rTA2wg',
  'videoId': '-GJgqIJsTME',
  'topLevelComment': {'kind': 'youtube#comment',
   'etag': 'MhL7ZvZJW8rHAXxXnistwmaBsBE',
   'id': 'Ugz64fenKxBqRkeabUB4AaABAg',
   'snippet': {'channelId': 'UC2utQmYluWGlhV-W0rTA2wg',
    'videoId': '-GJgqIJsTME',
    'textDisplay': 'Clovis’ teeth have seen better days',
    'textOriginal': 'Clovis’ teeth have seen better days',
    'authorDisplayName': '@gusblack3699',
    'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AIdro_lmSLLpoeaa6qOddYB7MqoFAXck8fG2GLGNXoM_3Fzmtw=s48-c-k-c0x00ffffff-no-rj',
    'authorChannelUrl': 'http://www.youtube.com/@gusblack3699',
    'authorChannelId': {'value': 'UCQB5sB2WRmw1JiPxJI0FlIA'},
    'canRate': True,
    'viewerRating': 'none',
    'likeCount': 0,
    'publishedAt': '2023-10-23T16:45:13Z',
    'updatedAt': '2023-10-23T16:45:13Z'}},
  'canReply': Tr

In [ ]:
df.head(10)

,author,updated_at,like_count,text,public
0,Action Bronson,2023-10-23T17:53:02Z,400,PARIS. WITH LOVE.,True
1,Daffers Little-Jones,2023-10-27T11:14:45Z,0,Best sequel since Terminator 2.,True
2,RyanK,2023-10-27T09:55:16Z,0,Clovis lives!,True
3,SnowMonkey AR,2023-10-27T09:49:56Z,0,"See, this is how you should visit Paris. Local...",True
4,Michael l,2023-10-27T09:21:01Z,0,Het Bourgondische leven zoals het bedoeld is.k...,True
5,Jacob Moore,2023-10-27T09:03:25Z,0,Clovis + Action in Paris is and always wil be ...,True
6,vntgzl,2023-10-27T08:56:02Z,0,PLEASE GO TO EVERY COUNTRY FOR THIS SERIES😩,True
7,J Williams,2023-10-27T07:20:23Z,1,nothing more authentic than a polish dude cook...,True
8,Victor Venet,2023-10-27T05:46:57Z,0,Something happend when those two guys are toge...,True
9,Ian V,2023-10-27T05:31:02Z,0,"I’ve been waiting for this for so long, God is...",True


## Sort by Likes and Get Top 10

In [ ]:
df.sort_values(by='like_count', ascending=False)[0:10]

,author,updated_at,like_count,text,public
682,@dirtsmegee,2023-10-23T15:05:54Z,1105,Action needs an 8 part series in every country...,True
585,@Nolanbrown480,2023-10-23T16:59:12Z,746,Clovis’ energy is absolutely unmatched. The pe...,True
0,@ActionBronsonMusic,2023-10-23T17:53:02Z,596,PARIS. WITH LOVE.,True
550,@ActionBronsonMusic,2023-10-23T17:53:02Z,596,PARIS. WITH LOVE.,True
687,@uknownutin3528,2023-10-23T15:01:03Z,538,One of the best food series on YouTube. LONG L...,True
696,@PinoPP,2023-10-23T12:30:09Z,278,One of the best and most authentic eating shows,True
636,@urbanprojectz,2023-10-23T15:59:02Z,178,This is a game changer for food/cooking/review...,True
660,@elmejortrabajodelmundo,2023-10-23T15:26:47Z,76,"""From Paris with Love"" makes me happy for real...",True
566,@yorkshireguy6671,2023-10-23T17:15:43Z,74,Action and Clovis in Paris always looks like t...,True
695,@cheng8841,2023-10-23T12:57:26Z,70,"""eight part series"" LETS GOOOOOO",True


#Pull all Comments for Multiple Videos

## 1. Get Comments Function

In [ ]:
import googleapiclient.discovery
import pandas as pd

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = dev

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)


def getcomments(video):
  request = youtube.commentThreads().list(
      part="snippet",
      videoId=video,
      maxResults=100
  )

  comments = []

  # Execute the request.
  response = request.execute()

  # Get the comments from the response.
  for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])

  while (1 == 1):
    try:
     nextPageToken = response['nextPageToken']
    except KeyError:
     break
    nextPageToken = response['nextPageToken']
    # Create a new request object with the next page token.
    nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
    # Execute the next request.
    response = nextRequest.execute()
    # Get the comments from the next response.
    for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])

  df2 = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])
  return df2

In [ ]:
df = getcomments('TYxqBTdOq24')
df

,author,updated_at,like_count,text,video_id,public
0,@3rjc11,2024-01-18T16:56:15Z,0,This was awesome thank you. Are these dashboar...,TYxqBTdOq24,True
1,@yaohun,2023-09-08T05:43:51Z,0,"Hi, sorry I am very new to SAP dashboard and h...",TYxqBTdOq24,True
2,@analyticswithadam,2023-05-10T06:12:50Z,3,Interested in thoughts on SAC compared with Po...,TYxqBTdOq24,True


## 2. For Loop for List of IDs

In [ ]:
df = pd.DataFrame()
for i in ['QOcP5OvSwlI','Lfzu74XDyco','TiS6vnju_mI','cYwioeHu_OU']:
  df2 = getcomments(i)
  df = pd.concat([df, df2])

In [ ]:
df

,author,updated_at,like_count,text,video_id,public
0,Artur Nawrocki,2023-10-27T06:28:00Z,0,SVG visuals has taken LinkedIn by storm. Too l...,QOcP5OvSwlI,True
1,Bostjan Brumec,2023-10-27T06:23:00Z,0,Very inspiring! Maybe it can be used to create...,QOcP5OvSwlI,True
2,Esteban Herrera,2023-10-27T04:12:19Z,0,As always excelent,QOcP5OvSwlI,True
3,Jyothi,2023-10-27T02:07:35Z,0,"Thanks Bas, for the video. Could you kindly te...",QOcP5OvSwlI,True
4,Anil Ameer,2023-10-26T19:42:54Z,0,"Take notes 📝 Microsoft, you are not listening...",QOcP5OvSwlI,True
...,...,...,...,...,...,...
440,sameer vishwakarma,2022-06-30T11:17:57Z,0,Cool,cYwioeHu_OU,True
441,Adam Sikorski,2022-06-30T11:13:33Z,0,Ok that looks dope,cYwioeHu_OU,True
442,Unq.Universalfacts,2022-06-30T11:09:16Z,0,Amazing.. Thanks a lot .. Can you plz share da...,cYwioeHu_OU,True
443,Ahmed Tareq,2022-06-30T11:04:25Z,0,🤩,cYwioeHu_OU,True


In [ ]:
df['video_id'].value_counts()

cYwioeHu_OU    445
Lfzu74XDyco    371
TiS6vnju_mI    100
QOcP5OvSwlI     50
Name: video_id, dtype: int64